In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
january = pd.read_parquet('../data/yellow/yellow_tripdata_2022-01.parquet')
february = pd.read_parquet('../data/yellow/yellow_tripdata_2022-02.parquet')

### Q1. Read the data for January. How many columns are there?

In [3]:
# There are 19 columns in the dataset
january.shape

(2463931, 19)

### Q2. What's the standard deviation of the trips duration in January?

In [4]:
january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
january['duration'] = january.tpep_dropoff_datetime - january.tpep_pickup_datetime
january['duration'] = january['duration'].dt.total_seconds() / 60

In [6]:
january['duration'].std()

46.44530513776499

### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
january.shape

(2463931, 20)

In [8]:
january = january[(january.duration >= 1) & (january.duration <= 60)]

In [9]:
january.shape

(2421440, 20)

In [10]:
2421440/2463931 * 100

98.27547930522405

### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [11]:
january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['duration']

In [13]:
january[categorical] = january[categorical].astype('str')

In [14]:
dv = DictVectorizer()

In [15]:
train_dicts = january[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [16]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [17]:
y_train = january['duration'].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)
yhat = lr.predict(X_train)

In [19]:
rmse = np.sqrt(mean_squared_error(y_train, yhat))
rmse

6.986190135953942

### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

In [20]:
february['duration'] = february.tpep_dropoff_datetime - february.tpep_pickup_datetime
february['duration'] = february['duration'].dt.total_seconds() / 60
february = february[(february.duration >= 1) & (february.duration <= 60)]
february[categorical] = february[categorical].astype('str')

In [21]:
valid_dicts = february[categorical].to_dict(orient='records')
X_valid = dv.transform(valid_dicts)
y_valid = february['duration'].values

In [22]:
yhat_valid = lr.predict(X_valid)

In [23]:
np.sqrt(mean_squared_error(y_valid, yhat_valid))

7.786389504099197